In [4]:
import sgkit as sg
import pandas as pd
import xarray as xr
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


In [6]:
def is_nonsynonymous(mutation_string):
    mutation_code = ds_logreg_ftsI.mutation.to_series().str.split().str[0]
    return (mutation_code.str[0] != mutation_code.str[-1]).values

ds_logreg = sg.load_dataset('../results/zarrs/Hinf_norm_bin_logreg_results.zarr/')
ds_logreg_ftsI = ds_logreg.sel(variants=((1688288 < ds_logreg.variant_position) & (ds_logreg.variant_position < 1690120)).compute())
ds_logreg_ftsI = ds_logreg_ftsI.sel(variants=is_nonsynonymous(ds_logreg_ftsI.mutation))

mask = ((ds_logreg_ftsI.call_genotype.loc['ID_1688312_A'] == 1) & (ds_logreg_ftsI.call_genotype.loc['ID_1688338_T'] == 1)).compute().squeeze()
ds_logreg_ftsI.sel(samples=mask).AMP.to_series().value_counts()['S'] 


32

In [ ]:
from itertools import product

def is_nonsynonymous(mutation_string):
    mutation_code = ds_logreg_ftsI.mutation.to_series().str.split().str[0]
    return (mutation_code.str[0] != mutation_code.str[-1]).values

ds_logreg = sg.load_dataset('results/zarrs/Hinf_norm_bin_logreg_results.zarr/')
ds_logreg_ftsI = ds_logreg.sel(variants=((1688288 < ds_logreg.variant_position) & (ds_logreg.variant_position < 1690120)).compute())
ds_logreg_ftsI = ds_logreg_ftsI.sel(variants=is_nonsynonymous(ds_logreg_ftsI.mutation))



variants = ds_logreg_ftsI.variants.values.tolist()

epsilon = 0.00001
ratios = {}
i = 0
for variant1, variant2 in product(variants, variants):
    if variant1 == variant2:
        pass
    else:
        mask = ((ds_logreg_ftsI.call_genotype.loc[variant1] == 1) & (ds_logreg_ftsI.call_genotype.loc[variant2] == 1)).compute().squeeze()
        subset_combi = ds_logreg_ftsI.sel(samples=mask)
        
        has_combi = mask.sum().values
        has_combi_R = subset_combi.AMP.to_series().value_counts().get('R', default=epsilon)
        has_combi_S = subset_combi.AMP.to_series().value_counts().get('S', default=epsilon)
        

        subset_combi_not = ds_logreg_ftsI.sel(samples=~mask)
        has_combi_not = (~mask).sum().values
        has_combi_not_R = subset_combi_not.AMP.to_series().value_counts().get('R', default=epsilon)
        has_combi_not_S = subset_combi_not.AMP.to_series().value_counts().get('S', default=epsilon)

        # print(has_combi, has_combi_not)
        odds_ratio = (has_combi_R/has_combi_S) / (has_combi_not_R/has_combi_not_S)
        ratios[(variant1, variant2)] = odds_ratio
        # print(has_combi_R+has_combi_S+has_combi_not_R+has_combi_not_S)
        # print(variant1, variant2)
        # print(has_combi_R, has_combi_S, has_combi_not_R, has_combi_not_S)
        # print(odds_ratio)
        # print('-------')

    if i == 40:
        break
    i += 1
        

ratios = pd.Series(ratios)


for variant in variants:
    mask = (ds_logreg_ftsI.call_genotype.loc[variant] == 1).values.squeeze()
    # print(mask)
    subset_has_mutation = ds_logreg_ftsI.sel(samples=mask)
    has_mutation_R = subset_has_mutation.AMP.to_series().value_counts().get('R', default=epsilon)
    has_mutation_S = subset_has_mutation.AMP.to_series().value_counts().get('S', default=epsilon)

    subset_has_not_mutation = ds_logreg_ftsI.sel(samples=~mask)
    has_not_mutation_R = subset_has_not_mutation.AMP.to_series().value_counts().get('R', default=epsilon)
    has_not_mutation_S = subset_has_not_mutation.AMP.to_series().value_counts().get('S', default=epsilon)

    odds_ratio = (has_mutation_R / has_mutation_S) / (has_not_mutation_R / has_not_mutation_S)
    print(variant)
    print(ds_logreg_ftsI.mutation.loc[variant].values)
    print(has_mutation_R, has_mutation_S, has_not_mutation_R, has_not_mutation_S)
    print(odds_ratio)
    print('-------------')

In [ ]:
ds_logreg_ftsI.call_genotype.loc['ID_1688990_A'].values

In [ ]:
ds_logreg_ftsI.mutation.loc['ID_1689466_T'].values

In [17]:
ds_linreg = sg.load_dataset('results/zarrs/Hinf_norm_mic_linreg_results.zarr/')
ds_linreg.effect.loc['ID_1688990_A'].values

array(1.03190352)

In [4]:
def is_nonsynonymous(mutation_string):
    # A20A (GCC/GCG) -> [A20A, (GCC/GCG)]
    mutation_code = mutation_string.to_series().str.split().str[0]
    # A ?= A
    return (mutation_code.str[0] != mutation_code.str[-1]).values

ds_linreg = sg.load_dataset('results/zarrs/Hinf_norm_mic_linreg_results.zarr/')
ds_linreg_ftsI = ds_linreg.sel(variants=((1688288 < ds_linreg.variant_position) & (ds_linreg.variant_position < 1690120)).compute())
ds_linreg_ftsI = ds_linreg_ftsI.sel(variants=is_nonsynonymous(ds_linreg_ftsI.mutation))

# the corresponding attributes somehow disappeared, so set them manually
ds_linreg_ftsI.attrs = {'contig_lengths' :[1830701],
                    'contigs' :['Hinf_Rd-KW20v3'],
                    'filters' :['PASS', 'LowQual']}

In [1]:
1690120 - 1688288

1832

In [69]:
ds_linreg_ftsI.mutation.loc['ID_1689667_C'].values

array('S152A (TCT/GCT)', dtype='<U15')

In [41]:
from sgkit.io.vcf import write_vcf

sg.save_dataset(ds_linreg_ftsI.reset_index('variants').reset_coords('variants'), 'test/nonsynonymous.zarr', auto_rechunk=True)
# ds_linreg_ftsI.reset_index('variants').reset_coords('variants')

In [6]:
from sgkit.io.vcf import zarr_to_vcf

zarr_to_vcf('test/nonsynonymous.zarr/', 'test/nonsynonymous.vcf')

In [ ]:
resistance_groups = ['ID_1688990_A', 'ID_1688990_G', 'ID_1688967_G', 'ID_1688954_A', 'ID_1688776_C', 'ID_1688616_A', 'ID_1688617_T', 'ID_1688571_T', 'ID_1688543_T', 'ID_1688543_C']

pp = sg.load_dataset('results/zarrs/Hinf_norm_mic_temp.zarr/')
mask = pp.variant_id.isin(resistance_groups)
pp.sel(variants=mask)



In [254]:
ds_linreg_ftsI['serotype'] = ds_linreg_ftsI['serotype'].to_series().replace('', 'NA')
ds_linreg_ftsI[['sample_id', 'serotype']].to_pandas().transpose().to_csv('test/serotype.csv', sep='\t', index=False, header=False)

In [246]:
ds_linreg_ftsI.serotpye

<xarray.DataArray 'serotpye' (samples: 262)>
array(['NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'Serotype f', 'NTHi', 'Serotype f', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'Serotype f', 'NTHi', 'NTHi',
       'Serotype f', 'NTHi', 'NTHi', 'Serotype f', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'Serotype f', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'Serotype f', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'Serotype f', 'Serotype e', 'Serotype f',
       'NTHi', 'NTHi', 'Serotype f', 'NTHi', 'NTHi', 'Serotype f', 'NTHi',
       'NTHi', 'Serotype f', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi', 'NTHi',
       'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
       'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
       'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
       'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
       'NA', 'NA', 'NA'], dtype=object)
Coordinates:
  * samples  (samples) int64 0 1 2 3 4 5 6 7 ... 254 255 256 257 258 259 260 261

In [234]:
ds_linreg_ftsI.sel(samples=(ds_linreg_ftsI.serotype == '').compute()).AMP.values

array(['S', 'S', 'S', 'S', 'S', 'S', 'R', 'S', 'R', 'R', 'S', 'S', 'S',
       'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S',
       'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'R', 'S', 'S',
       'S', 'R', 'S', 'S', 'S', 'R', 'R', 'S'], dtype=object)

In [229]:
sg.load_dataset('results/zarrs/Hinf_norm.zarr/').sample_id.values

array(['HLR-423', 'HLR-17', 'HLR-551', 'HLR-115', 'HLR-553', 'HLR-77',
       'HLR-181', 'HLR-507', 'HLR-56', 'HLR-409', 'HLR-451', 'HLR-133',
       'HLR-157', 'HLR-10', 'HLR-549', 'HLR-190', 'HLR-318', 'HLR-433',
       'HLR-391', 'HLR-110', 'HLR-492', 'HLR-3', 'HLR-532', 'HLR-544',
       'HLR-359', 'HLR-182', 'HLR-262', 'HLR-339', 'HLR-109', 'HLR-32',
       'HLR-287', 'HLR-539', 'HLR-97', 'HLR-515', 'HLR-272', 'HLR-174',
       'HLR-227', 'HLR-114', 'HLR-449', 'HLR-536', 'HLR-496', 'HLR-550',
       'HLR-450', 'HLR-484', 'HLR-461', 'HLR-36', 'HLR-93', 'HLR-105',
       'HLR-511', 'HLR-427', 'HLR-2', 'HLR-419', 'HLR-199', 'HLR-542',
       'HLR-194', 'HLR-446', 'HLR-292', 'HLR-111', 'HLR-414', 'HLR-280',
       'HLR-331', 'HLR-73', 'HLR-528', 'HLR-151', 'HLR-323', 'HLR-503',
       'HLR-137', 'HLR-216', 'HLR-349', 'HLR-344', 'HLR-134', 'HLR-51',
       'HLR-41', 'HLR-361', 'HLR-395', 'HLR-347', 'HLR-523', 'HLR-246',
       'HLR-202', 'HLR-254', 'HLR-385', 'HLR-139', 'HLR-314', 'HLR

In [233]:
ds_linreg_ftsI.sample_id.values

array(['HLR-106', 'HLR-109', 'HLR-10', 'HLR-110', 'HLR-111', 'HLR-112',
       'HLR-118', 'HLR-11', 'HLR-121', 'HLR-124', 'HLR-13', 'HLR-154',
       'HLR-155', 'HLR-156', 'HLR-159', 'HLR-162', 'HLR-17', 'HLR-19',
       'HLR-1', 'HLR-22', 'HLR-25', 'HLR-27', 'HLR-2', 'HLR-30', 'HLR-35',
       'HLR-36', 'HLR-38', 'HLR-40', 'HLR-44', 'HLR-46', 'HLR-47',
       'HLR-48', 'HLR-50', 'HLR-51', 'HLR-53', 'HLR-54', 'HLR-56',
       'HLR-57', 'HLR-59', 'HLR-61', 'HLR-67', 'HLR-68', 'HLR-71',
       'HLR-72', 'HLR-75', 'HLR-76', 'HLR-79', 'HLR-82', 'HLR-83',
       'HLR-84', 'HLR-85', 'HLR-86', 'HLR-8', 'HLR-90', 'HLR-93',
       'HLR-95', 'HLR-97', 'HLR-99', 'HLR-198', 'HLR-199', 'HLR-200',
       'HLR-202', 'HLR-203', 'HLR-205', 'HLR-206', 'HLR-210', 'HLR-212',
       'HLR-227', 'HLR-230', 'HLR-235', 'HLR-238', 'HLR-248', 'HLR-259',
       'HLR-271', 'HLR-272', 'HLR-273', 'HLR-274', 'HLR-275', 'HLR-276',
       'HLR-277', 'HLR-278', 'HLR-279', 'HLR-280', 'HLR-281', 'HLR-282',
       'HLR-283

In [11]:
import pandas as pd
hhh = pd.read_feather('../results/feather/Hinf_norm_mic.feather')



In [7]:
hhh[['samples', 'serotype']].transpose().to_csv('../test/serotype.csv', sep='\t', index=False, header=False)


# ds_linreg_ftsI[['sample_id', 'serotype']].to_pandas().transpose().to_csv('test/serotype.csv', sep='\t', index=False, header=False)